In [ ]:
import pdb

In [ ]:
if __name__ == '__main__':
    main()

In [ ]:
dfData.col

In [ ]:
stockName = 'SO.PA'

In [ ]:
dfData = loadStocks('stockprospects.json')
dfData = dfData.loc[dfData['stockname'] == stockName]
histoData = getStockData(stockName)
data=genIMGfromFile(graphEvolutionTitre(histoData, dfData.iloc[0]), 'img.png', scale=1.3, width=800, height=500)
histo = computeIchimoku(histoData)
startDate=date.today() - timedelta(days=60)
data=genIMGfromFile(graphIchimoku(
    dfData, histo[histo.index > startDate]
), 'img.png', scale=1.3, width=800, height=500)
mybot.sendPhoto(chat_id, open('img.png', 'rb'))
startDate=date.today() - timedelta(days=360)
data=genIMGfromFile(graphIchimoku(
    dfData, histo[histo.index > startDate]
), 'img.png', scale=1.3, width=800, height=500)
mybot.sendPhoto(chat_id, open('img.png', 'rb'))



In [ ]:
strOut